In [1]:
import sys
import os
import logging
import papermill as pm
import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import tensorflow.compat.v1 as tf
tf.get_logger().setLevel('ERROR') # only show error messages

if os.path.join('..', '..', 'recommenders') not in sys.path:
    sys.path.append(os.path.join('..', '..', 'recommenders'))

from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.models.deeprec.deeprec_utils import (
    prepare_hparams
)
from recommenders.datasets.amazon_reviews import download_and_extract, data_preprocessing
from recommenders.datasets.download_utils import maybe_download


# Locally import the model
from models.deeprec.models.sequential.din import DIN_RECModel as SeqModel


#from recommenders.models.deeprec.models.sequential.nextitnet import NextItNetModel

from recommenders.models.deeprec.io.sequential_iterator import SequentialIterator
#from recommenders.models.deeprec.io.nextitnet_iterator import NextItNetIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

import pickle as pkl
from recommenders.models.deeprec.deeprec_utils import load_dict

System version: 3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]
Tensorflow version: 2.9.0


In [2]:
yaml_file = '../../recommenders/models/deeprec/config/din.yaml'  

In [3]:
EPOCHS = 10
BATCH_SIZE = 400
RANDOM_SEED = SEED  # Set None for non-deterministic result

data_path = os.path.join("..", "..", "tests", "resources", "deeprec", "slirec")

In [4]:

# for test
train_file = os.path.join(data_path, r'train_data')
valid_file = os.path.join(data_path, r'valid_data')
test_file = os.path.join(data_path, r'test_data')
user_vocab = os.path.join(data_path, r'user_vocab.pkl')
item_vocab = os.path.join(data_path, r'item_vocab.pkl')
cate_vocab = os.path.join(data_path, r'category_vocab.pkl')
output_file = os.path.join(data_path, r'output.txt')

reviews_name = 'reviews_Movies_and_TV_5.json'
meta_name = 'meta_Movies_and_TV.json'
reviews_file = os.path.join(data_path, reviews_name)
meta_file = os.path.join(data_path, meta_name)
train_num_ngs = 4 # number of negative instances with a positive instance for training
valid_num_ngs = 4 # number of negative instances with a positive instance for validation
test_num_ngs = 9 # number of negative instances with a positive instance for testing
# sample_rate = 0.01 # sample a small item set for training and testing here for fast example
sample_rate = 1

input_files = [reviews_file, meta_file, train_file, valid_file, test_file, user_vocab, item_vocab, cate_vocab]

if not os.path.exists(train_file):
    download_and_extract(reviews_name, reviews_file)
    download_and_extract(meta_name, meta_file)
    data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)
    #### uncomment this for the NextItNet model, because it does not need to unfold the user history
    # data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs, is_history_expanding=False)

# data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)

# To analyze the dataset of Amazon

number of users: 123,960

number of movies: 50,024

number of categories: 164

number of historical actions for each user:

number of train samples: 1,325,653

number of test samples: 1,239,600

number of pos samples: 

number of neg samples:

In [5]:
hparams = prepare_hparams(yaml_file, 
                          embed_l2=0., 
                          layer_l2=0., 
                          learning_rate=0.001,  # set to 0.01 if batch normalization is disable
                          epochs=EPOCHS,
                          batch_size=BATCH_SIZE,
                          show_step=20,
                          MODEL_DIR=os.path.join(data_path, "model/"),
                          SUMMARIES_DIR=os.path.join(data_path, "summary/"),
                          user_vocab=user_vocab,
                          item_vocab=item_vocab,
                          cate_vocab=cate_vocab,
                          need_sample=True,
                          train_num_ngs=train_num_ngs, # provides the number of negative instances for each positive instance for loss computation.
            )

In [6]:
hparams.values()

{'use_entity': True,
 'use_context': True,
 'cross_activation': 'identity',
 'user_dropout': True,
 'dropout': [0.3, 0.3],
 'attention_dropout': 0.0,
 'load_saved_model': False,
 'fast_CIN_d': 0,
 'use_Linear_part': False,
 'use_FM_part': False,
 'use_CIN_part': False,
 'use_DNN_part': False,
 'init_method': 'tnormal',
 'init_value': 0.01,
 'embed_l2': 0.0,
 'embed_l1': 0.0,
 'layer_l2': 0.0,
 'layer_l1': 0.0,
 'cross_l2': 0.0,
 'cross_l1': 0.0,
 'reg_kg': 0.0,
 'learning_rate': 0.001,
 'lr_rs': 1,
 'lr_kg': 0.5,
 'kg_training_interval': 5,
 'max_grad_norm': 2,
 'is_clip_norm': 0,
 'dtype': 32,
 'optimizer': 'adam',
 'epochs': 10,
 'batch_size': 400,
 'enable_BN': True,
 'show_step': 20,
 'save_model': True,
 'save_epoch': 1,
 'write_tfevents': True,
 'train_num_ngs': 4,
 'need_sample': True,
 'embedding_dropout': 0.0,
 'EARLY_STOP': 10,
 'min_seq_length': 1,
 'slots': 5,
 'cell': 'SUM',
 'user_vocab': '../../tests/resources/deeprec/slirec/user_vocab.pkl',
 'item_vocab': '../../tests/r

In [9]:
input_creator = SequentialIterator
#### uncomment this for the NextItNet model, because it needs a special data iterator for training
#input_creator = NextItNetIterator

In [10]:
model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)

with Timer() as train_time:
    model = model.fit(train_file, valid_file, valid_num_ngs=valid_num_ngs) 

# valid_num_ngs is the number of negative lines after each positive line in your valid_file 
# we will evaluate the performance of model on valid_file every epoch
print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))

/home/caozheng/anaconda3/lib/python3.9/site-packages/recommenders-1.1.0-py3.9.egg/recommenders/models/deeprec/models/base_model.py:701: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  curr_hidden_nn_layer = tf.compat.v1.layers.batch_normalization(
2022-06-03 00:31:18.987810: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-03 00:31:19.043838: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-03 00:31:19.044024: I tensorflow/stream_ex

step 20 , total_loss: 1.5715, data_loss: 1.5715
step 40 , total_loss: 1.5763, data_loss: 1.5763
step 60 , total_loss: 1.5559, data_loss: 1.5559
step 80 , total_loss: 1.5045, data_loss: 1.5045
step 100 , total_loss: 1.5237, data_loss: 1.5237
step 120 , total_loss: 1.5685, data_loss: 1.5685
step 140 , total_loss: 1.5238, data_loss: 1.5238
step 160 , total_loss: 1.5029, data_loss: 1.5029
step 180 , total_loss: 1.5502, data_loss: 1.5502
step 200 , total_loss: 1.5264, data_loss: 1.5264
step 220 , total_loss: 1.5038, data_loss: 1.5038
step 240 , total_loss: 1.5354, data_loss: 1.5354
step 260 , total_loss: 1.5132, data_loss: 1.5132
step 280 , total_loss: 1.5112, data_loss: 1.5112
step 300 , total_loss: 1.4766, data_loss: 1.4766
step 320 , total_loss: 1.4373, data_loss: 1.4373
step 340 , total_loss: 1.4966, data_loss: 1.4966
step 360 , total_loss: 1.4150, data_loss: 1.4150
step 380 , total_loss: 1.4382, data_loss: 1.4382
step 400 , total_loss: 1.4698, data_loss: 1.4698
step 420 , total_loss: 1

eval valid at epoch 1: auc:0.7983,logloss:0.6678,mean_mrr:0.7267,ndcg@2:0.689,ndcg@4:0.7783,ndcg@6:0.7952,group_auc:0.7905
step 20 , total_loss: 1.0315, data_loss: 1.0315
step 40 , total_loss: 0.9942, data_loss: 0.9942
step 60 , total_loss: 1.0724, data_loss: 1.0724
step 80 , total_loss: 1.0451, data_loss: 1.0451
step 100 , total_loss: 1.0991, data_loss: 1.0991
step 120 , total_loss: 1.0817, data_loss: 1.0817
step 140 , total_loss: 1.0639, data_loss: 1.0639
step 160 , total_loss: 1.0790, data_loss: 1.0790
step 180 , total_loss: 1.0825, data_loss: 1.0825
step 200 , total_loss: 1.1222, data_loss: 1.1222
step 220 , total_loss: 1.0632, data_loss: 1.0632
step 240 , total_loss: 1.0763, data_loss: 1.0763
step 260 , total_loss: 1.0823, data_loss: 1.0823
step 280 , total_loss: 1.0155, data_loss: 1.0155
step 300 , total_loss: 1.0050, data_loss: 1.0050
step 320 , total_loss: 1.0620, data_loss: 1.0620
step 340 , total_loss: 1.0218, data_loss: 1.0218
step 360 , total_loss: 1.1594, data_loss: 1.1594

step 3280 , total_loss: 0.9914, data_loss: 0.9914
step 3300 , total_loss: 1.0637, data_loss: 1.0637
eval valid at epoch 2: auc:0.8228,logloss:0.7184,mean_mrr:0.757,ndcg@2:0.7264,ndcg@4:0.8045,ndcg@6:0.8181,group_auc:0.8174
step 20 , total_loss: 1.0124, data_loss: 1.0124
step 40 , total_loss: 0.9652, data_loss: 0.9652
step 60 , total_loss: 0.9943, data_loss: 0.9943
step 80 , total_loss: 0.9296, data_loss: 0.9296
step 100 , total_loss: 0.9176, data_loss: 0.9176
step 120 , total_loss: 0.9301, data_loss: 0.9301
step 140 , total_loss: 0.9425, data_loss: 0.9425
step 160 , total_loss: 0.9963, data_loss: 0.9963
step 180 , total_loss: 1.0203, data_loss: 1.0203
step 200 , total_loss: 0.9664, data_loss: 0.9664
step 220 , total_loss: 0.9034, data_loss: 0.9034
step 240 , total_loss: 0.9897, data_loss: 0.9897
step 260 , total_loss: 0.8319, data_loss: 0.8319
step 280 , total_loss: 0.9449, data_loss: 0.9449
step 300 , total_loss: 0.9869, data_loss: 0.9869
step 320 , total_loss: 0.8724, data_loss: 0.87

step 3240 , total_loss: 0.9778, data_loss: 0.9778
step 3260 , total_loss: 1.0067, data_loss: 1.0067
step 3280 , total_loss: 0.9838, data_loss: 0.9838
step 3300 , total_loss: 0.9762, data_loss: 0.9762
eval valid at epoch 3: auc:0.8329,logloss:0.7964,mean_mrr:0.7702,ndcg@2:0.7424,ndcg@4:0.8157,ndcg@6:0.828,group_auc:0.8284
step 20 , total_loss: 0.8572, data_loss: 0.8572
step 40 , total_loss: 0.8675, data_loss: 0.8675
step 60 , total_loss: 0.8927, data_loss: 0.8927
step 80 , total_loss: 0.9344, data_loss: 0.9344
step 100 , total_loss: 0.9494, data_loss: 0.9494
step 120 , total_loss: 0.9282, data_loss: 0.9282
step 140 , total_loss: 0.8659, data_loss: 0.8659
step 160 , total_loss: 0.8119, data_loss: 0.8119
step 180 , total_loss: 0.8190, data_loss: 0.8190
step 200 , total_loss: 0.8331, data_loss: 0.8331
step 220 , total_loss: 0.8972, data_loss: 0.8972
step 240 , total_loss: 0.8250, data_loss: 0.8250
step 260 , total_loss: 0.8855, data_loss: 0.8855
step 280 , total_loss: 0.9122, data_loss: 0.

...

step 3120 , total_loss: 0.8602, data_loss: 0.8602
step 3140 , total_loss: 0.7936, data_loss: 0.7936
step 3160 , total_loss: 0.8340, data_loss: 0.8340
step 3180 , total_loss: 0.9285, data_loss: 0.9285
step 3200 , total_loss: 0.9294, data_loss: 0.9294
step 3220 , total_loss: 0.8486, data_loss: 0.8486
step 3240 , total_loss: 0.7522, data_loss: 0.7522
step 3260 , total_loss: 0.8190, data_loss: 0.8190
step 3280 , total_loss: 0.8799, data_loss: 0.8799
step 3300 , total_loss: 0.7053, data_loss: 0.7053
eval valid at epoch 6: auc:0.8448,logloss:1.0981,mean_mrr:0.7864,ndcg@2:0.7617,ndcg@4:0.8291,ndcg@6:0.8402,group_auc:0.8418
step 20 , total_loss: 0.7282, data_loss: 0.7282
step 40 , total_loss: 0.6814, data_loss: 0.6814
step 60 , total_loss: 0.7712, data_loss: 0.7712
step 80 , total_loss: 0.7693, data_loss: 0.7693
step 100 , total_loss: 0.9051, data_loss: 0.9051
step 120 , total_loss: 0.8627, data_loss: 0.8627
step 140 , total_loss: 0.7383, data_loss: 0.7383
step 160 , total_loss: 0.7199, data_l

step 3080 , total_loss: 0.8273, data_loss: 0.8273
step 3100 , total_loss: 0.8022, data_loss: 0.8022
step 3120 , total_loss: 0.8809, data_loss: 0.8809
step 3140 , total_loss: 0.7719, data_loss: 0.7719
step 3160 , total_loss: 0.8705, data_loss: 0.8705
step 3180 , total_loss: 0.7603, data_loss: 0.7603
step 3200 , total_loss: 0.8438, data_loss: 0.8438
step 3220 , total_loss: 0.7939, data_loss: 0.7939
step 3240 , total_loss: 0.8383, data_loss: 0.8383
step 3260 , total_loss: 0.8704, data_loss: 0.8704
step 3280 , total_loss: 0.7547, data_loss: 0.7547
step 3300 , total_loss: 0.8852, data_loss: 0.8852
eval valid at epoch 7: auc:0.8468,logloss:1.2043,mean_mrr:0.7895,ndcg@2:0.7658,ndcg@4:0.8322,ndcg@6:0.8425,group_auc:0.8449
step 20 , total_loss: 0.7283, data_loss: 0.7283
step 40 , total_loss: 0.7179, data_loss: 0.7179
step 60 , total_loss: 0.7420, data_loss: 0.7420
step 80 , total_loss: 0.7333, data_loss: 0.7333
step 100 , total_loss: 0.7265, data_loss: 0.7265
step 120 , total_loss: 0.7573, data

step 3040 , total_loss: 0.7939, data_loss: 0.7939
step 3060 , total_loss: 0.8242, data_loss: 0.8242
step 3080 , total_loss: 0.8210, data_loss: 0.8210
step 3100 , total_loss: 0.7781, data_loss: 0.7781
step 3120 , total_loss: 0.7714, data_loss: 0.7714
step 3140 , total_loss: 0.7888, data_loss: 0.7888
step 3160 , total_loss: 0.8094, data_loss: 0.8094
step 3180 , total_loss: 0.8447, data_loss: 0.8447
step 3200 , total_loss: 0.7349, data_loss: 0.7349
step 3220 , total_loss: 0.7782, data_loss: 0.7782
step 3240 , total_loss: 0.7795, data_loss: 0.7795
step 3260 , total_loss: 0.7419, data_loss: 0.7419
step 3280 , total_loss: 0.7804, data_loss: 0.7804
step 3300 , total_loss: 0.8862, data_loss: 0.8862
eval valid at epoch 8: auc:0.8477,logloss:1.3159,mean_mrr:0.7914,ndcg@2:0.7684,ndcg@4:0.8333,ndcg@6:0.8439,group_auc:0.8461
step 20 , total_loss: 0.7944, data_loss: 0.7944
step 40 , total_loss: 0.7107, data_loss: 0.7107
step 60 , total_loss: 0.6760, data_loss: 0.6760
step 80 , total_loss: 0.7372, da

step 3000 , total_loss: 0.7405, data_loss: 0.7405
step 3020 , total_loss: 0.7502, data_loss: 0.7502
step 3040 , total_loss: 0.8475, data_loss: 0.8475
step 3060 , total_loss: 0.8076, data_loss: 0.8076
step 3080 , total_loss: 0.6590, data_loss: 0.6590
step 3100 , total_loss: 0.8089, data_loss: 0.8089
step 3120 , total_loss: 0.8164, data_loss: 0.8164
step 3140 , total_loss: 0.7684, data_loss: 0.7684
step 3160 , total_loss: 0.7552, data_loss: 0.7552
step 3180 , total_loss: 0.7392, data_loss: 0.7392
step 3200 , total_loss: 0.8311, data_loss: 0.8311
step 3220 , total_loss: 0.7427, data_loss: 0.7427
step 3240 , total_loss: 0.8356, data_loss: 0.8356
step 3260 , total_loss: 0.7827, data_loss: 0.7827
step 3280 , total_loss: 0.7707, data_loss: 0.7707
step 3300 , total_loss: 0.8522, data_loss: 0.8522
eval valid at epoch 9: auc:0.8484,logloss:1.4569,mean_mrr:0.793,ndcg@2:0.7693,ndcg@4:0.8345,ndcg@6:0.8451,group_auc:0.8467
step 20 , total_loss: 0.6409, data_loss: 0.6409
step 40 , total_loss: 0.7746,

step 2960 , total_loss: 0.7696, data_loss: 0.7696
step 2980 , total_loss: 0.7284, data_loss: 0.7284
step 3000 , total_loss: 0.7039, data_loss: 0.7039
step 3020 , total_loss: 0.7917, data_loss: 0.7917
step 3040 , total_loss: 0.7965, data_loss: 0.7965
step 3060 , total_loss: 0.6914, data_loss: 0.6914
step 3080 , total_loss: 0.8013, data_loss: 0.8013
step 3100 , total_loss: 0.7989, data_loss: 0.7989
step 3120 , total_loss: 0.7792, data_loss: 0.7792
step 3140 , total_loss: 0.6727, data_loss: 0.6727
step 3160 , total_loss: 0.8523, data_loss: 0.8523
step 3180 , total_loss: 0.6744, data_loss: 0.6744
step 3200 , total_loss: 0.7534, data_loss: 0.7534
step 3220 , total_loss: 0.7395, data_loss: 0.7395
step 3240 , total_loss: 0.7258, data_loss: 0.7258
step 3260 , total_loss: 0.8168, data_loss: 0.8168
step 3280 , total_loss: 0.7196, data_loss: 0.7196
step 3300 , total_loss: 0.7326, data_loss: 0.7326
eval valid at epoch 10: auc:0.8474,logloss:1.5456,mean_mrr:0.7922,ndcg@2:0.7686,ndcg@4:0.8337,ndcg@6

In [1]:
res_syn = model.run_eval(test_file, num_ngs=test_num_ngs)
print(res_syn)

NameError: name 'model' is not defined

In [11]:
model_best_trained = SeqModel(hparams, input_creator, seed=RANDOM_SEED)
path_best_trained = os.path.join(hparams.MODEL_DIR, "best_model")
print('loading saved model in {0}'.format(path_best_trained))
model_best_trained.load_model(path_best_trained)

/home/caozheng/anaconda3/lib/python3.9/site-packages/recommenders-1.1.0-py3.9.egg/recommenders/models/deeprec/models/base_model.py:701: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  curr_hidden_nn_layer = tf.compat.v1.layers.batch_normalization(


loading saved model in ../../tests/resources/deeprec/slirec/model/best_model


2022-06-03 01:53:58.730536: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-03 01:53:58.730693: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-03 01:53:58.730801: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-03 01:53:58.730938: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-03 01:53:58.731045: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [12]:
model_best_trained.run_eval(test_file, num_ngs=test_num_ngs)

{'auc': 0.8392,
 'logloss': 1.659,
 'mean_mrr': 0.6699,
 'ndcg@2': 0.6194,
 'ndcg@4': 0.6939,
 'ndcg@6': 0.7246,
 'group_auc': 0.836}